## Implementation of Blackjack to simulate various card counting strategies ##

In [358]:
import random
from timeit import default_timer as timer
import time

### 1. Implementation of Blackjack game with parameters for various rulesets ###

In [322]:
class Deck():
    def __init__(self, num_of_decks=1):
        self.deck = [i%52 for i in range(num_of_decks*52)]
        self.discard_deck = []
        
    def shuffle(self):
        random.shuffle(self.deck)
    
    def card_num_or_face(self, num):
        num_or_face = {0: 'Ace', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8',
                       8: '9', 9: '10', 10: 'Jack', 11: 'Queen', 12: 'King'}
        return num_or_face[num%13]
    
    def card_suit(self, num):  # Private methods
        suits = {0: 'Hearts', 1: 'Diamonds', 2: 'Clubs', 3: 'Spades'}
        return suits[num//13]
    
    def read_card(self, num):
        return self.card_num_or_face(num) + ' of ' + self.card_suit(num)

In [367]:
class Blackjack():
    # Allow various ruleset implementations here
    # -  allow 1 to 7 players
    # - between 1 and 8 decks is standard practice
    def __init__(self, num_of_decks=1, players=1, blackjack_payout=1.5, OTHER_RULESET_VARIABLES_HERE=0):
        self.deck_obj = Deck(num_of_decks)
        self.players = players
        self.blackjack_payout = blackjack_payout
        
    def __repr__(self):
        return ' '.join(map(str, self.deck_obj.deck))
    
    def shuffle(self, discard_top_card=True):
        self.deck_obj.shuffle()  # Shuffle deck
        if discard_top_card:  # Discard top card
            self.deck_obj.discard_deck.append(self.deck_obj.deck.pop())
        
        
    def display_deck(self):
        for card in self.deck_obj.deck:
            print(self.deck_obj.read_card(card))
            
    def next_card(self):
        return self.deck_obj.deck.pop()
    
    def is_blackjack(self, card1, card2):
        card1_value = card1%13
        card2_value = card2%13
        
        # Ace = 0, 10-King = 9-12
        if (card1_value == 0 and card2_value >= 9 and card2_value <= 12) or \
           (card2_value == 0 and card1_value >= 9 and card1_value <= 12):
            return True
        return False
    
    def get_player_value(self, player_hand, first_run=True):
        if self.is_blackjack(player_hand[0], player_hand[1]):
            return 'Blackjack'
        
        if first_run:
            self.player_values = []  # store as list so can have multiple if soft values
        
        player_value = 0
        for i, card in enumerate(player_hand):
            card_value = card%13
            if card_value == 0:  # Ace
                new_hand_1 = player_hand.copy()
                new_hand_2 = player_hand.copy()
                
                new_hand_1[i] = -1  # Ace as 1
                new_hand_2[i] = -2  # Ace as 11
                
                self.get_player_value(new_hand_1, first_run=False)
                self.get_player_value(new_hand_2, first_run=False)
                
                break
            else:
                if card == -1:  # Then this is soft 1 ace
                    card_value = 1
                elif card == -2:  # Then this is soft 11 ace
                    card_value = 11
                elif card_value >= 9:  # Then this is 10 or face card
                    card_value = 10
                else:  # Any other card increment by 1 for correct number
                    card_value += 1
                    
                player_value += card_value
        else:  # If there wasn't a 0 signifying un-declared Ace then append to list
            self.player_values.append(player_value)
            
        if first_run:
            # self.players_values = [value for value in self.player_values if value <=21]
            return self.player_values
        
    def get_player_input(self, player, human_player=True):
        if human_player:
            value = input('Player {} what is is your move? (HIT, STAND, DD, SPLIT, SURR):  '.format(player)).upper()
            print('You have chosen to {}'.format(value))

            if 'HIT' in value:
                return 0
            elif 'STAND' in value:
                return 1
            elif 'DD' in value:
                return 2
            elif 'SPLIT' in value:
                return 3
            elif 'SURR' in value:
                return 4
            else:
                print('Incorrect input. Please try again.')  
        else:  # ADD MACHINE HERE LATER
            raise ValueError('No computer player implemented')
            
    def play_hand(self, bet):
        # 1. Bet amount is logged into private variable
        self.__bet = bet
        # 2. Dealer gives 1 card to player (each player if multiple)
        self.player_hands = []
        for player in range(self.players):
            self.player_hands.append([self.next_card()])
            
        # 3. Dealer gives 1 card to themself face up
        self.dealer_hand = []
        self.dealer_hand.append(self.next_card())
        
        # 4. Dealer gives 2nd card to player (each player if multiple)
        for i, player in enumerate(range(self.players)):
            self.player_hands[i].append(self.next_card())
        
        # 5. Dealer gives 2nd card to themself: note should be face down so don't share [1] with players
        self.dealer_hand.append(self.next_card())

        # 6. Players are prompted on move
        for i, player in enumerate(range(self.players)):
            turn_complete = False
            
            while not turn_complete:
                player_value = self.get_player_value(self.player_hands[i])
                
                print('Player {}: You have a {} and a {} {}'.format(str(i+1),
                            self.deck_obj.read_card(self.player_hands[i][0]),
                            self.deck_obj.read_card(self.player_hands[i][1]), player_value))
                      
                print('Dealer has a ' + self.deck_obj.read_card(self.dealer_hand[0]))
                
                if player_value == 'Blackjack':  # player has blackjack
                    print('Blackjack!')
                    turn_complete = True
                elif min(player_value) > 21:  # player went bust
                    print('You went bust :(')
                    turn_complete = True
                else:                    
                    # Player action
                    player_action = self.get_player_input(player=i+1)

                    if player_action == 0:  # HIT
                        self.player_hands[i].append(self.next_card())
                        
                    elif player_action == 1:  # STAND
                        turn_complete = True
                    elif player_action == 2:  # DOUBLE DOWN
                        pass
                    elif player_action == 3:  # SPLIT
                        pass
                    elif player_action == 4:  # SURRENDER
                        pass
                    else:
                        print('Incorrect input. Please try again.')
                        
                
            print('Player {} turn complete\n'.format(i+1))
            
            time.sleep(1.5)
        # Put all cards into discard deck
        # Pay out odds/collect money
        # Go next turn
                        
                # dealer goes and calculate dealer_value
                # check player_value vs dealer_value

                        
                
            
                      
        


In [368]:
blackjack = Blackjack(num_of_decks=3, players=10)
blackjack


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51

In [369]:
blackjack.shuffle()
blackjack

42 25 25 17 16 6 4 34 41 0 10 13 50 38 31 7 2 26 42 13 12 27 48 16 3 13 33 45 43 25 48 36 12 35 38 18 22 45 30 17 20 45 47 4 9 48 44 50 31 37 6 43 2 32 17 14 11 32 15 35 37 34 28 5 21 9 19 23 49 26 36 23 20 39 0 15 32 14 33 3 15 27 41 22 19 8 24 0 30 29 8 1 49 39 42 37 43 28 5 27 51 5 34 46 36 26 1 29 21 24 6 4 18 11 12 14 51 38 19 28 40 47 40 10 47 39 51 50 29 11 16 23 44 18 3 49 20 33 41 44 9 2 46 21 7 24 22 30 1 31 40 8 7 46 10

In [370]:
blackjack.play_hand(bet=5)

Player 1: You have a Jack of Hearts and a 9 of Diamonds [19]
Dealer has a 8 of Hearts


KeyboardInterrupt: 

In [ ]:
# Turn order
# 1. Place your bet
# 2. Dealer puts top card to discard pile if first card of deck IRRELEVANT but implement anyway in case it changes deck structure
# 3. Dealer gives 1 card face up to every player from left to right
# 4. Dealer gives 1 card to himself face up
# 5. Dealer gives 2nd card to everyone face up
# 6. Dealer gives 2nd card to himself face down
# 7. Player chooses to Hit, Stand, Double Down, Split, Surrender (in some rulesets and only on first hand)
# 8. Once you stand (or double down and get 1 more card)
# Blackjack pays 3-2
# If both get blackjack,usually a push (money back). But sometimes dealer always wins on tie! 2 card blackjack anyway


# Double down can do up to 100%. Some games only 100%. You get 1 card then dealer goes
# Can usually split face cards and 10 but some casinos allow only if identical face
# Ace and 10 after split is counted as a non blackjack 21
# Hitting split aces is usually not allowed
# Sometimes a double after a split is not allowed
# If the dealers face up card is an ace you can pay for insurance. Pay extra max of half current bet and paid out 2:1
# if the dealer's 2nd card is a 10

# Rule of whether dealer has to hit soft 17 (s17): ace and 16. Wikipedia says 0.2% in player's favour if they do
# Tend to stand on hard 17 (h17)


# More decks is more of a house advantage. Fewer decks is higher chance of player Blackjack. Lower chance of bj-bj


In [200]:
blackjack.play_hand(bet=5)

In [190]:
blackjack.shuffle()
blackjack


17 29 33 18 5 8 27 7 31 33 6 39 51 44 47 36 45 3 35 24 34 43 34 10 19 10 42 46 49 22 11 1 41 18 21 44 27 43 17 0 48 20 44 42 38 2 26 32 19 15 40 8 15 22 16 48 36 49 13 14 36 50 10 24 0 3 34 12 13 50 37 39 7 35 28 37 4 23 5 23 28 9 50 46 6 32 35 17 6 42 16 21 43 45 25 40 20 13 9 23 24 22 27 1 2 0 45 41 41 1 19 26 7 4 2 25 16 47 51 30 31 40 49 12 11 4 39 32 11 3 15 20 8 29 47 14 31 38 30 18 12 28 9 26 38 5 33 29 21 25 48 30 14 37 51

In [191]:
blackjack.deck_obj.discard_deck

[46]

In [184]:
blackjack = Blackjack(num_of_decks=1)
blackjack.display_deck()

Ace of Hearts
2 of Hearts
3 of Hearts
4 of Hearts
5 of Hearts
6 of Hearts
7 of Hearts
8 of Hearts
9 of Hearts
10 of Hearts
Jack of Hearts
Queen of Hearts
King of Hearts
Ace of Diamonds
2 of Diamonds
3 of Diamonds
4 of Diamonds
5 of Diamonds
6 of Diamonds
7 of Diamonds
8 of Diamonds
9 of Diamonds
10 of Diamonds
Jack of Diamonds
Queen of Diamonds
King of Diamonds
Ace of Clubs
2 of Clubs
3 of Clubs
4 of Clubs
5 of Clubs
6 of Clubs
7 of Clubs
8 of Clubs
9 of Clubs
10 of Clubs
Jack of Clubs
Queen of Clubs
King of Clubs
Ace of Spades
2 of Spades
3 of Spades
4 of Spades
5 of Spades
6 of Spades
7 of Spades
8 of Spades
9 of Spades
10 of Spades
Jack of Spades
Queen of Spades
King of Spades
